<a href="https://colab.research.google.com/github/talha-0/MedicalMind/blob/main/Medical_Mind_For_Patients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MEDICAL MIND
A personalised medical assistant

## Downloading Dependencies

In [ ]:
!pip install -qU pinecone-client langchain openai sqlalchemy tiktoken cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00


## Setting up environment


In [ ]:
from getpass import getpass
import os

In [ ]:
os.environ["PINECONE_API_KEY"] = getpass("Pinecone API Key:")

Pinecone API Key:··········


In [ ]:
os.environ["PINECONE_ENVIRONMENT"] = getpass("Pinecone Environment:")

Pinecone Environment:··········


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


## External source
Automatic Glossary of Clinical Terminology: a Large-Scale Dictionary of Biomedical Definitions Generated from Ontological Knowledge

In [ ]:
!git clone https://huggingface.co/datasets/FremyCompany/AGCT-Dataset

Cloning into 'AGCT-Dataset'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (16/16), 5.19 KiB | 1.04 MiB/s, done.


In [ ]:
!git clone https://github.com/abachaa/MedQuAD.git

Cloning into 'MedQuAD'...
remote: Enumerating objects: 11310, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 11310 (delta 8), reused 8 (delta 3), pack-reused 11292
Receiving objects: 100% (11310/11310), 11.01 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (6806/6806), done.
Updating files: 100% (11277/11277), done.


### Preprocess the data


In [ ]:
import pandas as pd

AGCT = pd.read_fwf("AGCT-Dataset/train.txt",header=None)

In [ ]:
AGCT

,0
0,"""zinc supplement"" refers to a medication that ..."
1,"""fit post core to tooth"" refers to a dental re..."
2,"""abdominal tenderness absent"" refers to a clin..."
3,"""divergence paralysis"" refers to a condition c..."
4,"""normal active range of hip extension"" refers ..."
...,...
421211,"""product containing zotarolimus"" refers to a m..."
421212,"""meiotic cell"" refers to a type of cell that u..."
421213,"""phage type battersea"" refers to a classificat..."
421214,"""diaphragmatic"" refers to anything that is rel..."


In [ ]:
AGCT.columns

Int64Index([0], dtype='int64')

In [ ]:
AGCT[['title','context']] = AGCT[0].str.split(' refers to ',1,expand=True)

<ipython-input-9-8d22cf2a3a48>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  AGCT[['title','context']] = AGCT[0].str.split(' refers to ',1,expand=True)


In [ ]:
AGCT.drop([0],axis=1,inplace=True)

In [ ]:
AGCT

,title,context
0,"""zinc supplement""",a medication that contains zinc and/or zinc co...
1,"""fit post core to tooth""",a dental restoration procedure that involves t...
2,"""abdominal tenderness absent""",a clinical finding of the absence of pain or d...
3,"""divergence paralysis""",a condition characterized by the inability to ...
4,"""normal active range of hip extension""",the normal range of movement that the hip join...
...,...,...
421211,"""product containing zotarolimus""",a medicinal product that acts as an immunosupp...
421212,"""meiotic cell""",a type of cell that undergoes a specific type ...
421213,"""phage type battersea""","a classification of bacteriophages, which are ..."
421214,"""diaphragmatic""",anything that is related to or located in the ...


## Initializing DB and creating tables

In [ ]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text, Date, ForeignKey, ForeignKeyConstraint

# Create a SQLAlchemy engine with an in-memory SQLite database
engine = create_engine("sqlite:///:memory:")
metadata = MetaData()

# Define the 'patientInfo' table
patient_info_table = Table(
    'patientInfo',
    metadata,
    Column('username', String(255), nullable=False, primary_key=True),
    Column('patientAge', Integer, nullable=True),
    Column('patientGender', String(10), nullable=True)
)

# Define the 'records' table
medical_records_table = Table(
    'medicalRecords',
    metadata,
    Column('username', String(255), ForeignKey('patientInfo.username'), nullable=False),
    Column('diagnosisText', Text),
    Column('diagnosisDate', Date),
    Column('bloodPressure', String(20), nullable=True),
    Column('cholesterol', String(20), nullable=True),
    Column('heartRate', Integer, nullable=True),
    ForeignKeyConstraint(['username'], ['patientInfo.username'])
)


metadata.create_all(engine)

## Inserting some dummy values

In [ ]:
from datetime import datetime

with engine.begin() as connection:
  patient_info_insert = patient_info_table.insert().values(username='Bob', patientAge=30, patientGender='Male')
  result = connection.execute(patient_info_insert)
  username = result.inserted_primary_key[0]

  # Insert an observation for the heart patient into the 'records' table
  medical_records_insert = medical_records_table.insert().values(
        username='Bob',
        diagnosisText='Heart condition diagnosed',
        diagnosisDate=datetime(2023, 12, 1),
        bloodPressure='bp=120/80',
        cholesterol='cholesterol=200',
        heartRate=75
    )
  connection.execute(medical_records_insert)

  print("Observations added for the heart patient:")
  print(f"Patient ID: {username}")

Observations added for the heart patient:
Patient ID: Bob


## Setting up a user

In [ ]:
USERNAME = 'Bob'

In [ ]:
RECORDS = []
with engine.begin() as connection:
  stmt = medical_records_table.select().where(username == USERNAME)
  stmt = connection.execute(stmt)
  RECORDS.append(stmt.all())
print(RECORDS)

[[('Bob', 'Heart condition diagnosed', datetime.date(2023, 12, 1), 'bp=120/80', 'cholesterol=200', 75)]]


In [ ]:
INFO = []
with engine.begin() as connection:
  stmt = patient_info_table.select().where(username == USERNAME)
  stmt = connection.execute(stmt)
  INFO.append(stmt.all())
print(INFO)

[[('Bob', 30, 'Male')]]


## Initializing llm

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0
)

## Create records tool

In [ ]:
from langchain.tools import BaseTool,Tool

class RecordTool(BaseTool):
  name = "Patients Record"
  description = "Useful to get patient's records. Each record includes diagnosis, bp, cholestrol and heartrate."

  def _run(self,query:str)->list:
      return RECORDS

  def _arun(self):
      raise NotImplementedError("RecordTool does not support async")

class PersonaTool(BaseTool):
  name = "Patients Info"
  description = "Useful to get patient's info. Includes name, age and gender"

  def _run(self,query:str)->list:
      return INFO

  def _arun(self):
      raise NotImplementedError("PersonaTool does not support async")

## AGENT with memory
Just testing

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(memory_key="chat_history",llm=llm,return_messages=True)

tools = [
    RecordTool(),
    PersonaTool()
]

In [ ]:
from langchain.agents import initialize_agent,AgentType

agent = initialize_agent(
    agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools = tools,
    llm=llm,
    verbose=True,
    early_stopping_method = 'generate',
    memory = memory
)

In [ ]:
agent("List all my past records")



> Entering new AgentExecutor chain...
```json
{
    "action": "Patients Record",
    "action_input": "List all records"
}
```
Observation: [[('Bob123', 'Heart condition diagnosed', datetime.date(2023, 12, 1))]]
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Your last recorded medical event was a diagnosis of a heart condition on December 1, 2023."
}
```

> Finished chain.


{'input': 'List all my past records',
 'chat_history': [HumanMessage(content='List all my past records'),
  AIMessage(content='Your last recorded medical event was a diagnosis of a heart condition on December 1, 2023.')],
 'output': 'Your last recorded medical event was a diagnosis of a heart condition on December 1, 2023.'}

## Pinecone

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

In [ ]:
import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENVIRONMENT"),  # next to api key in console
)

index_name = "medical-mind"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric="dotproduct",
        dimension=1536
    )
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 1.0,
 'namespaces': {'': {'vector_count': 100992}},
 'total_vector_count': 100992}

In [ ]:
text_field = 'text'

vectorstore = Pinecone(
    index,
    embeddings.embed_query,
    text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:62: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Indexing

In [ ]:
!pip install canopy-sdk

In [ ]:
!canopy

Usage: canopy [OPTIONS] COMMAND [ARGS]...

  CLI for Pinecone Canopy. Actively developed by Pinecone.
  To use the CLI, you need to have a Pinecone account.
  Visit https://www.pinecone.io/ to sign up for free.

Options:
  -v, --version  Show the version and exit.
  -h, --help     Show this message and exit.

Commands:
  new       Create a new Pinecone index that will be used by Canopy.
  upsert    Upload local data files to the Canopy service.
  start     Start the Canopy server.
  chat      Debugging tool for chatting with the Canopy RAG service.
  health    Check if the Canopy server is running and healthy.
  stop      Stop the Canopy server.
  api-docs  Open the Canopy server docs.


In [ ]:
!canopy start

In [ ]:
contexts = AGCT['context'].to_list()
max([len(context) for context in contexts])

579

In [ ]:
from tqdm.auto import tqdm

batch_size = 64
for i in tqdm(range(0,len(AGCT),batch_size)):
  i_end = min(len(AGCT),i+batch_size)
  # Get batch of data
  batch = AGCT.iloc[i:i_end]
  # Unique ids
  ids = [f"{x.name}" for _,x in batch.iterrows()]
  # Text to embed
  texts = [x['context'] for _,x in batch.iterrows()]
  # Embed Text
  embeds = embeddings.embed_documents(texts)
  # Metadata to store
  metadata = [
      {
          'text': x['context'],
          'title': x['title']
      } for _,x in batch.iterrows()
  ]
  index.upsert(vectors=zip(ids,embeds,metadata))

In [ ]:
vectorstore.similarity_search(" i have heart problems",k=3)

[Document(page_content='the condition in which the heart muscle is damaged or impaired following exposure to a medication, chemotherapy, radiotherapy, or other substances, resulting in various types and severity of heart problems.', metadata={'title': '"cardiotoxicity"'}),
 Document(page_content='a type of long-term heart disease caused by insufficient blood supply to the heart muscle due to narrowed or blocked coronary arteries. It is typically associated with chest pain or discomfort, shortness of breath, and fatigue.', metadata={'title': '"chronic ischemic heart disease"'}),
 Document(page_content='an irregular or disrupted blood flow within the heart or the blood vessels surrounding it. This condition can be caused by a variety of factors, including congenital heart defects, valvular diseases, or damage to the heart muscle due to heart attack or other conditions. Symptoms can include shortness of breath, chest pain, and fatigue, and diagnosis requires a detailed medical evaluation,

In [ ]:
os.environ["INDEX_NAME"]=index_name

--------------------------------------------------------------------------------

# 1. DB Agent
Trying out first agent

In [ ]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase

In [ ]:
db = SQLDatabase(engine)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(memory_key="chat_history",llm=llm)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.2 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS # Facebook AI Similarity Search

embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY
)# ada-002
docs = [
    Document(
      page_content=title,
      metadata = {"refers_to": context}
    ) for title, context in zip(AGCT['title'][:50], AGCT['context'][:50])
]
vector_db = FAISS.from_documents(docs, embeddings)
retriever = vector_db.as_retriever()

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
Use it to give second opinion and diagnosis.
Input to this tool should be the user question.
"""
retriever_tool = create_retriever_tool(
    retriever, name="biomedical_similar_examples", description=tool_description
)
custom_tool_list = [retriever_tool]

In [ ]:
custom_suffix = """
I always direct to the most suitable specialist and also give a second opinion
and diagnosis.
If the username is given I should get medical records from the database.
I should then get the similar examples I know.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS, # CHAT_CONVERSATIONAL_REACT_DESCRIPTION not supported so no memory
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)

In [ ]:
agent.run(
    "I am Bob123 I am feeling chest pain"
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_schema` with `medical_records`


Error: table_names {'medical_records'} not found in database
Invoking: `sql_db_list_tables` with ``


medicalRecords, patientInfo
Invoking: `sql_db_schema` with `medicalRecords, patientInfo`



CREATE TABLE "medicalRecords" (
	username INTEGER NOT NULL, 
	"diagnosisText" TEXT, 
	"diagnosisDate" DATE, 
	FOREIGN KEY(username) REFERENCES "patientInfo" (username)
)

/*
3 rows from medicalRecords table:
username	diagnosisText	diagnosisDate
Bob123	Heart condition diagnosed	2023-12-01
*/


CREATE TABLE "patientInfo" (
	username VARCHAR(255) NOT NULL, 
	PRIMARY KEY (username)
)

/*
3 rows from patientInfo table:
username
Bob123
*/
Invoking: `sql_db_query` with `SELECT diagnosisText, diagnosisDate FROM medicalRecords WHERE username = 'Bob123' ORDER BY diagnosisDate DESC LIMIT 10`


[('Heart condition diagnosed', '2023-12-01')]
Invoking: `biomedical_similar_examples` with `{'query': 'chesst pain'}`


[Doc

"Based on your medical records, you were diagnosed with a heart condition on 2023-12-01. Given your current complaint of chest pain, it's important that you seek immediate medical attention. Chest pain can be a symptom of serious conditions such as heart disease. \n\nIn terms of similar symptoms, some examples include difficulty balancing when bending, late effect of injury to blood vessels of thorax, abdomen and/or pelvis, and abdominal tenderness. However, these are just examples and may not be directly related to your condition. Please consult with a healthcare professional for an accurate diagnosis."

In [ ]:
print(agent.agent.prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessage(content='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\

In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

________________________________________________________________________

# 2. React agent
Can be improved not following all instructions

In [ ]:

medical_mind = initialize_agent(
    agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools = tools,
    llm=llm,
    verbose=True,
    early_stopping_method = 'generate',
    memory = memory,
    prefix = custom_prefix,
    suffix = custom_suffix
)

In [ ]:
medical_mind("I am not feeling very well i am feeling tightness in my chest")



> Entering new AgentExecutor chain...
```json
{
    "action": "biomedical_similar_examples",
    "action_input": "Patient is not feeling well and is experiencing tightness in the chest."
}
```
Observation: [Document(page_content='a sensation of discomfort or pain in the chest, often described as pressure or tightness. It may be caused by a variety of underlying conditions, including but not limited to cardiovascular or respiratory diseases, gastrointestinal disorders, or musculoskeletal issues. Chest discomfort may require further evaluation by a healthcare provider to determine the underlying cause and appropriate treatment.', metadata={'title': '"chest discomfort"'}), Document(page_content='a symptom characterized by a feeling of fullness, tightness, or discomfort in the chest. It can be associated with various medical conditions affecting the cardiovascular, respiratory, or gastrointestinal systems, such as angina, myocardial infarction, pulmonary embolism, or gastroesophageal ref

'The tightness in your chest could be due to a variety of conditions. It could be related to cardiovascular issues like angina or myocardial infarction, respiratory conditions, gastrointestinal disorders, or even anxiety. This symptom is generally considered a warning sign of potential serious health problems and requires prompt medical attention. Please seek immediate medical care.'

In [ ]:
medical_mind("Which type of doctor should i visit")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Given your symptoms of chest tightness, it would be appropriate to seek immediate medical attention. Ideally, you should go to an emergency department. If that's not possible, you should contact a healthcare professional who can evaluate your condition urgently. This could be a general practitioner or an internist, but they may refer you to a cardiologist given the nature of your symptoms."
}
```

> Finished chain.


{'input': 'Which type of doctor should i visit',
 'chat_history': [HumanMessage(content='I am not feeling very well i am feeling tightness in my chest'),
  AIMessage(content="The sensation of discomfort or pain in your chest, often described as pressure or tightness, can be caused by a variety of underlying conditions. Given your previous diagnosis of a heart condition, it's particularly important to take this symptom seriously. It could be related to cardiovascular issues such as angina or myocardial infarction. However, it could also be related to other systems, such as respiratory or gastrointestinal disorders. This symptom generally requires prompt medical attention. Please seek immediate medical care."),
  HumanMessage(content='Which type of doctor should i visit'),
  AIMessage(content="Given your symptoms of chest tightness, it would be appropriate to seek immediate medical attention. Ideally, you should go to an emergency department. If that's not possible, you should contact a 

In [ ]:
medical_mind("I also feel out of breath at moments")



> Entering new AgentExecutor chain...
```json
{
    "action": "Patients Record",
    "action_input": "Requesting patient's records to understand the history of symptoms and any pre-existing conditions."
}
```
Observation: [[('Bob123', 'Heart condition diagnosed', datetime.date(2023, 12, 1), 'bp=120/80', 'cholesterol=200', 75)]]
Thought:```json
{
    "action": "biomedical_similar_examples",
    "action_input": "Patient is experiencing chest tightness and moments of breathlessness. Patient has a diagnosed heart condition as of December 1, 2023. Blood pressure is 120/80 and cholesterol is 200. Patient's age is 75."
}
```
Observation: [Document(page_content='a finding in which the healthcare provider has identified systolic blood pressure over 180 mmHg or diastolic blood pressure over 120 mmHg during a physical examination of a patient. This is an abnormally high blood pressure reading that requires prompt evaluation and treatment to prevent further complications.', metadata={'title': '"

{'input': 'I also feel out of breath at moments',
 'chat_history': [HumanMessage(content='I am not feeling very well i am feeling tightness in my chest'),
  AIMessage(content="The sensation of discomfort or pain in your chest, often described as pressure or tightness, can be caused by a variety of underlying conditions. Given your previous diagnosis of a heart condition, it's particularly important to take this symptom seriously. It could be related to cardiovascular issues such as angina or myocardial infarction. However, it could also be related to other systems, such as respiratory or gastrointestinal disorders. This symptom generally requires prompt medical attention. Please seek immediate medical care."),
  HumanMessage(content='Which type of doctor should i visit'),
  AIMessage(content="Given your symptoms of chest tightness, it would be appropriate to seek immediate medical attention. Ideally, you should go to an emergency department. If that's not possible, you should contact a

In [ ]:
medical_mind.agent.llm_chain.prompt

# 3. Final Agent
 Rag With Memory, Patient persona and records

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(memory_key="chat_history",llm=llm,return_messages=True)
tool_description = """
Use it to give second opinion and diagnosis.
Input to this tool should be the query with relevant patients records.
"""
tools = [
    RecordTool(),
    PersonaTool(),
    Tool(
        name='biomedical_similar_examples',
        func = vectorstore.similarity_search,
        description=tool_description
    )
]

In [ ]:
from langchain.agents import AgentExecutor,ZeroShotAgent,AgentType
from langchain.chains import LLMChain
import langchain.agents as agents
custom_prefix = """
First the Agent gets the patient persona. \
Then the agent gets the medical records of the patient. \
Lastly it gets the similar examples for diagnosis. \
After all the processing Agent recommends the specific doctor to visit and also \
gives its diagnosis.

You have access to the following tools:
"""
custom_suffix = """
Always make sure to get patient info and records before giving any diagnosis.
Begin!

{chat_history}
Question: {input}
{agent_scratchpad}
"""
prompt = agents.StructuredChatAgent.create_prompt(
    tools,
    prefix=custom_prefix,
    suffix=custom_suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

medical_mind = agents.StructuredChatAgent(llm_chain=llm_chain, tools=tools, verbose=True)

medical_mind_chain = AgentExecutor.from_agent_and_tools(
    agent=medical_mind, tools=tools, verbose=True, memory=memory
)

In [ ]:
medical_mind_chain.run("I am not feeling very well i am feeling tightness in my chest")



> Entering new AgentExecutor chain...
Thought: The patient is complaining about chest tightness. I need to get more information about the patient. I will use the Patients Info tool to get the patient's information.
Action:
```
{
  "action": "Patients Info",
  "action_input": {"query": "patient's information"}
}
```
Observation: [[('Bob', 30, 'Male')]]
Thought:The patient is a 30-year-old male named Bob. Now, I need to get his medical records to understand his medical history better. I will use the Patients Record tool for this. 
Action:
```
{
  "action": "Patients Record",
  "action_input": {"query": "Bob's records"}
}
```
Observation: [[('Bob', 'Heart condition diagnosed', datetime.date(2023, 12, 1), 'bp=120/80', 'cholesterol=200', 75)]]
Thought:The patient, Bob, has a history of heart condition diagnosed on 1st December 2023. His blood pressure was 120/80, cholesterol level was 200, and heart rate was 75. Given his symptoms of chest tightness and his medical history, it's possible 

"Given your symptoms and medical history, it's possible that your heart condition might be causing the chest tightness. However, it could also be due to other issues like gastrointestinal or respiratory problems. I recommend you to visit a cardiologist for a thorough examination."

In [ ]:
medical_mind_chain.run("I also feel out of breath at times")



> Entering new AgentExecutor chain...
Thought: The patient is experiencing chest tightness and shortness of breath. These symptoms could be related to a heart condition. I need to get more information about the patient's medical history to make a more accurate assessment. 
Action:
```
{
  "action": "Patients Info",
  "action_input": {"query": "patient's information"}
}
```
Observation: [[('Bob', 30, 'Male')]]
Thought:The patient is a 30-year-old male named Bob. Now, I need to get his medical records to understand his medical history better. 
Action:
```
{
  "action": "Patients Record",
  "action_input": {"query": "Bob's medical records"}
}
```
Observation: [[('Bob', 'Heart condition diagnosed', datetime.date(2023, 12, 1), 'bp=120/80', 'cholesterol=200', 75)]]
Thought:The patient, Bob, has a diagnosed heart condition. His blood pressure is 120/80, which is within the normal range, and his cholesterol level is 200, which is borderline high. His heart rate is 75, which is also within th

"Given your symptoms and medical history, it's possible that you might have a condition such as hypertrophic cardiomyopathy without obstruction or chronic ischemic heart disease. Both conditions can cause chest tightness and shortness of breath. However, these are just possibilities and you should seek immediate medical attention. I recommend you to visit a cardiologist for a thorough examination."

In [ ]:
while True:
  user_input = input()
  result = medical_mind_chain(user_input)
  print(result['output'])



---

